# Classifying Satellite Imagery with GNNs
## CIS700

### Matt Graber

### Obtaining the data

First, we will obtain the data. We will be attempting to classify the Corrected Reflectance (True Color) Suomi NPP / VIIRS product available from the Global Imagery Browse Services (GIBS) API. To label the data for training and testing, we will be using the Clear Sky Confidence (Day) Suomi NPP / VIRRS product and the Land Water Map (OSM) (from Open Street Map served through GIBS).

Corrected Reflectance (True Color) Layer example: https://gibs.earthdata.nasa.gov/wms/epsg4326/best/wms.cgi?version=1.3.0&service=WMS&request=GetMap&format=image/png&STYLE=default&bbox=-90,-180,90,180CRS=EPSG:4326&HEIGHT=600&WIDTH=600&TIME=2022-05-01&layers=VIIRS_SNPP_CorrectedReflectance_TrueColor

Clear Sky Confidence Layer Example: https://gibs.earthdata.nasa.gov/wms/epsg4326/best/wms.cgi?version=1.3.0&service=WMS&request=GetMap&format=image/png&STYLE=default&bbox=-90,-180,90,180CRS=EPSG:4326&HEIGHT=600&WIDTH=600&TIME=2022-05-01&layers=VIIRS_SNPP_Clear_Sky_Confidence_Day

Land Water Map Layer Example: https://gibs.earthdata.nasa.gov/wms/epsg4326/best/wms.cgi?version=1.3.0&service=WMS&request=GetMap&format=image/png&STYLE=default&bbox=-90,-180,90,180CRS=EPSG:4326&HEIGHT=600&WIDTH=600&TIME=2022-05-01&layers=OSM_Land_Water_Map

In [1]:
import cv2
import csv
import numpy as np
import datetime
import time
import os
from skimage import io
from PIL import Image as plimg

In [2]:
layers = ["VIIRS_SNPP_CorrectedReflectance_TrueColor", "VIIRS_SNPP_Clear_Sky_Confidence_Day"]
startdate = datetime.date(2022,5,1)
enddate = datetime.date(2022,5,6)
img_extent_step = 5
resolution = 128

for layer in layers:
    print("Downloading {} images...".format(layer))
    layer_outdir = os.path.join(os.getcwd(), "images", layer)
    currentdate = startdate

    while currentdate < enddate:
        outdir = os.path.join(layer_outdir, str(currentdate))
        
         # Create directory if it doesn't exist yet
        if not os.path.exists(outdir):
            os.makedirs(outdir, exist_ok=True)
            
        print("Downloading images for {}...".format(currentdate))
        for longitude in range(-180,180,img_extent_step):
            for latitude in range(-90,90,img_extent_step):
                extents = "{0},{1},{2},{3}".format(latitude, longitude,
                                                latitude + img_extent_step,
                                                longitude + img_extent_step)
                outfilepath = os.path.join(outdir,'{0}_{1}_{2}.png'.format(layer, currentdate, extents))
                # Skip any files that have already been downloaded
                # (this enables quick resumption if connection errors are encountered).
                # put this in a while-loop in case there's a connection error and
                # the download for something needs to be retried
                while not os.path.exists(outfilepath) or cv2.imread(outfilepath) is None:
                    # Construct image URL.
                    url = 'https://gibs.earthdata.nasa.gov/wms/epsg4326/best/wms.cgi?\
version=1.3.0&service=WMS&request=GetMap&\
format=image/png&STYLE=default&bbox={0}&CRS=EPSG:4326&\
HEIGHT={3}&WIDTH={3}&TIME={1}&layers={2}'.format(extents, currentdate, layer, resolution)
                    
                    # Occasionally we get an error from a momentary dropout of internet connection or something.
                    # This try-except should 
                    try:
                        # Request and save image
                        img = plimg.fromarray(io.imread(url))
                        img.save(outfilepath)
                    except:
                        print("Error encountered, retrying")
                        time.sleep(5)

        currentdate += datetime.timedelta(1)

# OSM_Land_Water_Map is a static layer, meaning that we don't need to re-download it for every day.
layer = "OSM_Land_Water_Map"
print("Downloading {} images...".format(layer))
outdir = os.path.join(os.getcwd(), "images", "{}".format(layer))

# Create directory if it doesn't exist yet
if not os.path.exists(outdir):
    os.mkdir(outdir)

for longitude in range(-180,180,img_extent_step):
    for latitude in range(-90,90,img_extent_step):
        extents = "{0},{1},{2},{3}".format(latitude, longitude,
                                        latitude + img_extent_step,
                                        longitude + img_extent_step)
        outfilepath = os.path.join(outdir,'{0}_{1}.png'.format(layer, extents))
        # Skip any files that have already been downloaded
        # (this enables quick resumption if connection errors are encountered)
        while not os.path.exists(outfilepath) or cv2.imread(outfilepath) is None:
            # Construct image URL.
            url = 'https://gibs.earthdata.nasa.gov/wms/epsg4326/best/wms.cgi?\
version=1.3.0&service=WMS&request=GetMap&\
format=image/png&STYLE=default&bbox={0}&CRS=EPSG:4326&\
HEIGHT={2}&WIDTH={2}&layers={1}'.format(extents, layer, resolution)
            # Occasionally we get an error from a momentary dropout of internet connection or something.
            # This try-except should 
            try:
                # Request and save image
                img = plimg.fromarray(io.imread(url))
                img.save(outfilepath)
            except:
                print("Error encountered, retrying")
                time.sleep(5)

### Labeling the data
Next, we will label each Corrected Reflectance image to indicate whether it contains land, water, and/or clouds to use as the training data for the neural networks. We will do this based on the percentages of colors in the corresponding images from Clear Sky Confidence and Land Water Map.

In [3]:
labeled_data_filename = "labeled_data.csv"

layer_to_label_path = os.path.join("images","VIIRS_SNPP_CorrectedReflectance_TrueColor")
clear_sky_layer_path = os.path.join("images", "VIIRS_SNPP_Clear_Sky_Confidence_Day")
land_water_map_path = os.path.join("images", "OSM_Land_Water_Map")
lw_filelist = os.listdir(land_water_map_path)

resolution = 128
pixel_count = resolution ** 2
# Exclude any images where 40% or more of the image is "no data"
nodata_threshold = pixel_count * 0.6

# dict for memoization of land water map results, since this is a static layer
lw_results = {}

with open(labeled_data_filename, 'w') as f:
    writer = csv.writer(f)
    writer.writerow(["filepath", "weather", "terrain"])
    for date in os.listdir(layer_to_label_path):
        print("Labeling for date {}...".format(date))
        co_re_datepath = os.path.join(layer_to_label_path, date)
        cl_sk_datepath = os.path.join(clear_sky_layer_path, date)
        co_re_filelist = os.listdir(co_re_datepath)
        cl_sk_filelist = os.listdir(cl_sk_datepath)
        for i in range(len(co_re_filelist)):
            # these directories should be ordered the same
            co_re_imgpath = os.path.join(co_re_datepath, co_re_filelist[i])
            cl_sk_imgpath = os.path.join(cl_sk_datepath, cl_sk_filelist[i])
            
            csv_row = [co_re_imgpath]

            # First, check if the corrected reflectance image is in an area of "no data"
            # i.e. it's all or mostly pure black.
            # We want to skip these images.
            co_re_img = cv2.imread(co_re_imgpath, 0) # use 0 flag to read grayscale
            if cv2.countNonZero(co_re_img) < nodata_threshold:
                continue

            # Next, check if the image is mostly cloudy or not cloudy.
            # In this layer, the reddish color (the higher pixel value) corresponds to clear skies,
            # and the whiteish color (the lower pixel value) corresponds to cloudy skies.
            cl_sk_img = cv2.imread(cl_sk_imgpath, 0)
            
            # If there are more dark-colored pixels than light-colored pixels, then it's not cloudy.
            if cv2.countNonZero(cv2.inRange(cl_sk_img, 0, 127)) > cv2.countNonZero(cv2.inRange(cl_sk_img, 128, 255)):
                # clear skies
                csv_row.append("clear")
            else:
                # cloudy skies
                csv_row.append("cloudy")

            # Finally, check if the image is mostly land or water.
            lw_imgpath = os.path.join(land_water_map_path, lw_filelist[i])
            if lw_imgpath in lw_results.keys():
                csv_row.append(lw_results[lw_imgpath])
            else:
                lw_img = cv2.imread(lw_imgpath, 0)
                # If there are more light-colored pixels than dark-colored pixels, then its mostly water
                if cv2.countNonZero(cv2.inRange(lw_img, 128, 128)) > cv2.countNonZero(cv2.inRange(lw_img, 75, 75)):
                    lw_results[lw_imgpath] = "water"
                else:
                    lw_results[lw_imgpath] = "land"
                csv_row.append(lw_results[lw_imgpath])
            writer.writerow(csv_row)

print("Labeling complete!")


Labeling for date 2022-05-01...
Labeling for date 2022-05-02...
Labeling for date 2022-05-03...
Labeling for date 2022-05-04...
Labeling for date 2022-05-05...
Labeling complete!


### Convolutional Neural Network (non-graph)
Now that our data is labeled, we can start by training a baseline non-graph convolutional neural network classifier. This follows the tutorial at https://learnopencv.com/multi-label-image-classification-with-pytorch.

In [2]:
import torch as tr
from torch import nn
from torch.utils.data import DataLoader
from torch.nn import functional as F
from torchvision import transforms, models
import random
import csv
import ast

# use a seed to keep the training and testing datasets the same for all models we test
random_seed = 44

# convert from strings to int representation of the labels
def encode(label):
    if label in ['clear', 'water']:
        return 0
    else: # cloudy, land
        return 1

# Used for obtaining the training/testing data
def load_data(filename):
    imgs = []
    weather = []
    terrain = []
    with open(filename) as datacsv:
        reader = csv.DictReader(datacsv)
        for row in reader:
            imgs.append(row["filepath"])
            weather.append(row["weather"])
            terrain.append(row["terrain"])
    shufflelist = list(zip(imgs, weather, terrain))
    random.Random(random_seed).shuffle(shufflelist)
    imgs, weather, terrain = zip(*shufflelist)
    imgs, weather, terrain = list(imgs), list(weather), list(terrain)
    # split into training and test data (use 60% for training, 40% for testing)
    split_size = int(0.6 * len(imgs))
    training_data = (imgs[:split_size], weather[:split_size], terrain[:split_size])
    testing_data = (imgs[split_size:], weather[split_size:], terrain[split_size:])
    return training_data, testing_data

class CorrectedReflectanceDataset(tr.utils.data.Dataset):
    def __init__(self, data):
        self.imgs, self.weather, self.terrain = data
    
    def __getitem__(self, idx):
        # take the data sample by its index
        img = plimg.open(self.imgs[idx])
        # ditch the transparency
        img = img.convert('RGB')

        # Normalize the image and convert to tensor
        # First calculate the mean and standard deviation of pixel values
        npimg = np.array(img)
        mean = np.mean(npimg, axis=(0,1))
        std = np.std(npimg, axis=(0,1))
        transform2 = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean, std)])
        img = transform2(img)
        """transform2 = transforms.Compose([transforms.ToTensor()])
        img = transform2(img)"""
        # return the image and the associated labels
        dict_data = {
            'img': img,
            'labels': {
                'weather': encode(self.weather[idx]),
                'terrain': encode(self.terrain[idx])
            }
        }
        return dict_data
    
    def __len__(self):
        return len(self.imgs)
    
class MultiOutputModel(nn.Module):
    def __init__(self, n_weather_classes, n_terrain_classes):
        super().__init__()
        self.base_model = models.mobilenet_v2().features  # take the model without classifier
        last_channel = models.mobilenet_v2().last_channel # size of the layer before the classifier
 
        # the input for the classifier should be two-dimensional, but we will have
        # [<batch_size>, <channels>, <width>, <height>]
        # so, let's do the spatial averaging: reduce <width> and <height> to 1
        self.pool = nn.AdaptiveAvgPool2d((1, 1))
 
        # create separate classifiers for our outputs
        self.weather = nn.Sequential(
            nn.Dropout(p=0.2),
            nn.Linear(in_features=last_channel, out_features=n_weather_classes)
        )
        self.terrain = nn.Sequential(
            nn.Dropout(p=0.2),
            nn.Linear(in_features=last_channel, out_features=n_terrain_classes)
        )
    
    def forward(self, x):
        x = self.base_model(x)
        x = self.pool(x)
    
        # reshape from [batch, channels, 1, 1] to [batch, channels] to put it into classifier
        x = tr.flatten(x, start_dim=1)
    
        return {
            'weather': self.weather(x),
            'terrain': self.terrain(x)
        }

    def get_loss(self, net_output, ground_truth):
        weather_loss = F.cross_entropy(net_output['weather'], ground_truth['weather'])
        terrain_loss = F.cross_entropy(net_output['terrain'], ground_truth['terrain'])
        
        loss = weather_loss + terrain_loss
        return loss, {'weather': weather_loss, 'terrain': terrain_loss}

def calculate_batch_metrics(predicted, target):
    weather_correct_count = 0
    terrain_correct_count = 0
    
    # should all be same length
    for i in range(len(predicted['weather'])):
        if predicted['weather'][i] == target['weather'][i]:
            weather_correct_count += 1
        if predicted['terrain'][i] == target['terrain'][i]:
            terrain_correct_count += 1
    weather_acc = weather_correct_count / len(predicted['weather'])
    terrain_acc = terrain_correct_count / len(predicted['terrain'])
    return weather_acc, terrain_acc

filename = "labeled_data.csv"
training_data, testing_data = load_data(filename)

N_epochs = 25
batch_size = 16
device = 'cpu'#'cuda:0'
outfilename = 'losses_results.csv'


training_dataset = CorrectedReflectanceDataset(training_data)
train_dataloader = DataLoader(training_dataset, batch_size=batch_size)
   
 
model = MultiOutputModel(n_weather_classes=2, n_terrain_classes=2).to(device)
 
optimizer = tr.optim.Adam(model.parameters())

# keep track of training time
start_time = time.time()
with open(outfilename, "w+") as f:
    csvwriter = csv.writer(f)
    csvwriter.writerow(["epoch","total loss", "weather loss", "terrain loss"])
    for epoch in range(N_epochs):
        print("Epoch: {}/{}".format(epoch+1, N_epochs))
        # Set to training mode
        model.train()
        # Loss within the epoch
        train_loss = 0.0
        train_losses = {'weather': 0.0, 'terrain': 0.0}
        for step, batch in enumerate(train_dataloader):
            # Clean existing gradients
            optimizer.zero_grad()

            img = batch['img']
            target_labels = batch['labels']
            
            # Forward pass - compute outputs on input data using the model
            outputs = model(img.to(device))
            target_labels['weather'] = target_labels['weather'].to(device)
            target_labels['terrain'] = target_labels['terrain'].to(device)
            # Compute loss
            loss_train, losses_train = model.get_loss(outputs, target_labels)
            #total_loss += loss_train.item()
            #batch_accuracy_weather, batch_accuracy_terrain = calculate_batch_metrics(outputs, target_labels)

            train_loss += float(loss_train)
            train_losses['weather'] += float(losses_train['weather'])
            train_losses['terrain'] += float(losses_train['terrain'])

            # Backpropagate the gradients
            loss_train.backward()
            # Update the parameters
            optimizer.step()
            
            #print("Batch number: {:03d}, Training: Loss: {:.4f}, Weather Accuracy: {:.4f}, Terrain Accuracy: {:.4f}".format(i, loss_train, batch_accuracy_weather.item(), batch_accuracy_terrain.item()))
        csvwriter.writerow([epoch, train_loss, train_losses['weather'], train_losses['terrain']])
        f.flush()
print("Total training time: %s seconds" % (time.time() - start_time))

Epoch: 1/25
Epoch: 2/25
Epoch: 3/25
Epoch: 4/25
Epoch: 5/25
Epoch: 6/25
Epoch: 7/25
Epoch: 8/25
Epoch: 9/25
Epoch: 10/25
Epoch: 11/25
Epoch: 12/25
Epoch: 13/25
Epoch: 14/25
Epoch: 15/25
Epoch: 16/25
Epoch: 17/25
Epoch: 18/25
Epoch: 19/25
Epoch: 20/25
Epoch: 21/25
Epoch: 22/25
Epoch: 23/25
Epoch: 24/25
Epoch: 25/25
Total training time: 6437.483986377716 seconds


We'll evaluate the accuracy, precision, recall, and f1 score for this model.

In [3]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

testing_dataset = CorrectedReflectanceDataset(testing_data)
test_dataloader = DataLoader(testing_dataset, batch_size=batch_size)

# put the model into evaluation mode
model.eval()

for label_type in ['weather', 'terrain']:

    print(label_type, "results:")
    for dataset_type in ["Training", "Testing"]:
        # initialize storage for ground truth and predicted labels
        predicted_all = []
        gt_all = []
        # go over all the images
        dataloader = train_dataloader if dataset_type == "Training" else test_dataloader
        for batch in dataloader:
            images = batch["img"]
            # we're going to build the confusion matrix for "weather" predictions
            gts = batch["labels"][label_type]
            target_labels = {label_type: gts.to(device)}
        
            # get the model outputs
            output = model(images.to(device))
        
            # get the most confident prediction for each image
            _, predicteds = output[label_type].cpu().max(1)
        
            predicted_all.extend(
                prediction.item() for prediction in predicteds
            )
            gt_all.extend(
                gt.item() for gt in gts
            )
        
        accuracy = accuracy_score(gt_all, predicted_all)
        precision = precision_score(gt_all, predicted_all)
        recall = recall_score(gt_all, predicted_all)
        f1 = f1_score(gt_all, predicted_all)

        print("  {}:".format(dataset_type))
        print("    Accuracy:", accuracy)
        print("    Precision:", precision)
        print("    Recall:", recall)
        print("    F1-score:", f1)
print("Total runtime:", str(time.time() - start_time))

weather results:
  Training:
    Accuracy: 0.7250231982678627
    Precision: 0.7212930202637504
    Recall: 0.9900662251655629
    F1-score: 0.8345738742091553
  Testing:
    Accuracy: 0.7214749536178108
    Precision: 0.714975845410628
    Recall: 0.9929553840992955
    F1-score: 0.8313439123718579
terrain results:
  Training:
    Accuracy: 0.3159604082895144
    Precision: 0.2912980537236609
    Recall: 0.9907002188183808
    F1-score: 0.45021752641392176
  Testing:
    Accuracy: 0.3200371057513915
    Precision: 0.297036858588292
    Recall: 0.9887730553327987
    F1-score: 0.4568358651352353
Total runtime: 6735.020060539246
